## 1) Inspiration

I chose this dataset because heart disease is a leading cause of death, and understanding its risk factors can help with prevention. This topic is also personally significant to me, as some of my family members have experienced heart disease. The dataset includes biological, lifestyle, and socioeconomic factors of heart disease, allowing for a holistic analysis of their impact. Exploring these relationships can provide valuable insights for public health and disease prevention.

## 2) Stakeholders

The key stakeholders include healthcare professionals, policymakers, researchers, and the general public. Doctors and public health officials can use the findings to improve prevention strategies, policymakers can develop better health initiatives, researchers can explore new correlations, and individuals can gain awareness of risk factors to make informed lifestyle choices.

## 3) Task and Metrics

A classification task will be implemented on the data since the response is binary. The evaluation metrics used will be accuracy, recall, precision, and ROC-AUC score. Accuracy measures overall correctness, recall assesses the model’s ability to detect actual heart disease cases, precision evaluates how many predicted cases are truly positive, and ROC-AUC measures how well the model distinguishes between individuals with and without heart disease. These metrics ensure a balanced assessment of the model’s performance.

## 4) Data

Access to the data set at this link: https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset/data

The data is from the Behavioral Risk Factor Surveillance System collected by the CDC annually. This data is from 2015, and it has been already cleaned. More information on how the data was cleaned up can be found here: https://www.kaggle.com/code/alexteboul/heart-disease-health-indicators-dataset-notebook#Link-to-Dataset-Output-Heart-Disease-Health-Indicators-Dataset 

- Dataset has 253680 observations and 22 variables
- There are 22 numerical variables, 0 categorical variables
- Most of them are binary, some ordinal. More information in the table below.
- Response variable: HeartDiseaseorAttack
- Predictor variables: All other variables are potential predictor variables

| Variable Name          | Meaning                                         | Question / category                                      |
|------------------------|------------------------------------------------|------------------------------------------------------|
| HeartDiseaseorAttack  | History of heart disease or heart attack       | Ever told you had a heart attack or heart disease?  |
| HighBP               | High blood pressure diagnosis                   | Ever told you have high blood pressure?            |
| HighChol             | High cholesterol diagnosis                      | Ever told you have high cholesterol?               |
| CholCheck            | Time since last cholesterol check               | How long since you last had cholesterol checked?   |
| BMI                  | Body Mass Index (BMI)                           | BMI (calculated with weight and height) * 100                    |
| Smoker               | Smoked at least 100 cigarettes in lifetime      | Have you smoked at least 100 cigarettes in your lifetime? |
| Stroke               | Ever diagnosed with a stroke                    | Ever told you had a stroke?                        |
| Diabetes             | Ever diagnosed with diabetes                    | Ordinal with 0 for no diabetes or only during pregnancy, 1 for pre-diabetes or borderline diabetes, 2 for yes diabetes                      |
| PhysActivity         | Engagement in physical activity                 | Do you engage in physical activity?                |
| Fruits               | Consumes fruits regularly                       | How often do you eat fruits?                       |
| Veggies              | Consumes vegetables regularly                   | How often do you eat vegetables?                   |
| HvyAlcoholConsump    | Heavy alcohol consumption                       | How often do you have more than 2 drinks per day?  |
| AnyHealthcare        | Has any form of healthcare coverage             | Do you have any health insurance coverage?         |
| NoDocbcCost          | Could not see a doctor due to cost              | Was there a time you couldn't see a doctor due to cost? |
| GenHlth              | Self-reported general health status             | Would you say your health is excellent, very good, good, fair, or poor? (1 is Excellent, 5 is Poor) |
| MentHlth             | Number of days with poor mental health          | How many days was your mental health not good in the past 30 days? (scale of 1 - 30)|
| PhysHlth             | Number of days with poor physical health        | How many days was your physical health not good in the past 30 days? (scale of 1 - 30)|
| DiffWalk             | Difficulty walking or climbing stairs           | Do you have difficulty walking or climbing stairs? |
| Sex                  | Sex of the respondent                           | Are you male or female? (0 for female, 1 for male)                            |
| Age                  | Age group                                       | Fourteen-level age category (ordinal: 1 = 18-24, increasing in 5-year increments)                            |
| Education            | Highest level of education attained            | Ordinal with 1 being never attended school or kindergarten up to 6 being college 4 years or more |
| Income               | Household income level                          | Ordinal with 1 being less than $10,000 all the way up to 8 being $75,000 or more       |


## 5) Prediction

In [2]:
#| echo: false
import pandas as pd
import numpy as np
data = pd.read_csv ('heart disease data.csv')

In [3]:
#| echo: false
#import
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


#Prepare data 
y = data['HeartDiseaseorAttack']
X = data.drop (columns=['HeartDiseaseorAttack'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

### Part 1: Linear unregularized model

I started with HighBP as my first variable because it is one of the three key variables CDC identified as a key risk factor for Heart disease. In this initial exploration, I used statsmodels to create a linear model. Since there is strong class imbalance in this dataset with 229,787 respondents not having had heart disease while 23,893 having had heart disease (ratio is 1:10), accuracy scores were really high, while precision and recall scores were really low. 


In [7]:
#| echo: false

import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

results = []

selected_predictors = ["HighBP"]
previous_predictors = []  

for predictor in X.columns:
    if predictor not in selected_predictors:
        selected_predictors.append(predictor)
    
    formula = "HeartDiseaseorAttack ~ " + " + ".join(selected_predictors)
    
    model = smf.logit(formula=formula, data=pd.concat([X_train, y_train], axis=1)).fit(disp=False)
    
    train_pred_prob = model.predict(X_train)
    test_pred_prob = model.predict(X_test)
    
    train_pred = (train_pred_prob >= 0.5).astype(int)
    test_pred = (test_pred_prob >= 0.5).astype(int)

    train_accuracy = accuracy_score(y_train, train_pred)
    test_accuracy = accuracy_score(y_test, test_pred)
    train_precision = precision_score(y_train, train_pred)
    test_precision = precision_score(y_test, test_pred)
    train_recall = recall_score(y_train, train_pred)
    test_recall = recall_score(y_test, test_pred)
    train_roc_auc = roc_auc_score(y_train, train_pred_prob)
    test_roc_auc = roc_auc_score(y_test, test_pred_prob)

    new_predictor = predictor if predictor not in previous_predictors else "Initial Predictor"
    results.append({
        "Added Predictor": new_predictor,
        "Train Accuracy": train_accuracy,
        "Test Accuracy": test_accuracy,
        "Train Precision": train_precision,
        "Test Precision": test_precision,
        "Train Recall": train_recall,
        "Test Recall": test_recall,
        "Train ROC-AUC": train_roc_auc,
        "Test ROC-AUC": test_roc_auc
    })
    
    previous_predictors.append(predictor)

results_df = pd.DataFrame(results)
display (results_df)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_

,Added Predictor,Train Accuracy,Test Accuracy,Train Precision,Test Precision,Train Recall,Test Recall,Train ROC-AUC,Test ROC-AUC
0,HighBP,0.905767,0.906004,0.000000,0.000000,0.000000,0.000000,0.677781,0.675763
1,HighChol,0.905767,0.906004,0.000000,0.000000,0.000000,0.000000,0.725159,0.722661
2,CholCheck,0.905767,0.906004,0.000000,0.000000,0.000000,0.000000,0.727052,0.724182
3,BMI,0.905767,0.906004,0.000000,0.000000,0.000000,0.000000,0.729081,0.727365
4,Smoker,0.905767,0.906004,0.000000,0.000000,0.000000,0.000000,0.748666,0.746114
5,Stroke,0.905338,0.905708,0.483750,0.488654,0.067716,0.067729,0.770362,0.765947
6,Diabetes,0.905639,0.905353,0.495599,0.476858,0.076553,0.071294,0.779315,0.775738
7,PhysActivity,0.906422,0.905964,0.529701,0.498175,0.062016,0.057245,0.782738,0.778232
8,Fruits,0.906403,0.905964,0.528603,0.498182,0.062330,0.057454,0.782806,0.778313
9,Veggies,0.906413,0.906122,0.528565,0.505415,0.063376,0.058713,0.783240,0.778934


The first predictor (HighBP) had the following test and train performances: 

In [14]:
#| echo: false
display (results_df.head(1))

,Added Predictor,Train Accuracy,Test Accuracy,Train Precision,Test Precision,Train Recall,Test Recall,Train ROC-AUC,Test ROC-AUC
0,HighBP,0.905767,0.906004,0.0,0.0,0.0,0.0,0.677781,0.675763


Train and test accuracy scores are extremely high while the Precision and Recall scores are at 0.  

**Some Takeaways**

- The largest increases in Test ROC-AUC occur after adding General Health (GenHlth), Difficulty Walking (DiffWalk), and Age, suggesting they are key predictors. This aligns with expectations, as older individuals are more likely to experience difficulty walking and poorer overall health, both of which are associated with a higher risk of heart disease.

### Part 2: Polynomial and regularized model
To adjust for this data imbalance and have a more accurate model, a regularized model using Ridge was used. Polynomial terms of order 2 were created. The 5 most important features and unimportant features were found, and there were no variables where the coefficient was 0.

In [9]:
#| echo: false
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegressionCV

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

feature_names = poly.get_feature_names_out(X.columns)

logistic_cv = LogisticRegressionCV(
    Cs=np.logspace(-4, 4, 10),  
    cv=10,  
    penalty='l2',  
    solver='saga', 
    max_iter=5000,  
    class_weight='balanced', 
    random_state=42
)
logistic_cv.fit(X_train_poly, y_train)

logistic_coef_df = pd.DataFrame({"Feature": feature_names, "Coefficient": logistic_cv.coef_.flatten()})
logistic_coef_df["Abs Coefficient"] = logistic_coef_df["Coefficient"].abs()

top_5_logistic_features = logistic_coef_df.nlargest(5, "Abs Coefficient")
bottom_5_logistic_features = logistic_coef_df.nsmallest(5, "Abs Coefficient")

print("Top 5 Most Important Features:")
display(top_5_logistic_features)

print("Top 5 Least Important Features:")
display(bottom_5_logistic_features)

Top 5 Most Important Features:


,Feature,Coefficient,Abs Coefficient
18,Age,0.533586,0.533586
13,GenHlth,0.326186,0.326186
0,HighBP,0.243489,0.243489
17,Sex,0.238312,0.238312
1,HighChol,0.234336,0.234336


Top 5 Least Important Features:


,Feature,Coefficient,Abs Coefficient
69,CholCheck Veggies,-0.000061,0.000061
180,Veggies PhysHlth,0.000090,0.000090
68,CholCheck Fruits,0.000144,0.000144
40,HighBP Education,-0.000162,0.000162
231,PhysHlth^2,-0.000180,0.000180


In [10]:
#| echo: false
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

y_test_pred_prob = logistic_cv.predict_proba(X_test_poly)[:, 1]  
y_test_pred = logistic_cv.predict(X_test_poly) 

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_pred_prob)

print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test ROC-AUC Score:", test_roc_auc)

Test Accuracy: 0.7559523809523809
Test Precision: 0.24836385271369074
Test Recall: 0.7877961836863074
Test ROC-AUC Score: 0.8462591657567866


**Takeaways from the most/least important features** 

- As expected, Age and General Health (GenHlth) are among the strongest predictors of heart disease.
- Consistent with CDC research, High Blood Pressure (HighBP) and High Cholesterol (HighChol) are strongly associated with heart disease risk.
- Interestingly, the model suggests that men are more likely to develop heart disease than women. Some research indicates that hormones like estrogen and progesterone, which are higher in women, may help protect blood vessels and reduce heart disease risk.

**Comparing test performance with part 1**

- While test accuracy has decreased, the new model provides a more realistic estimate and remains solid at 76%.
- The drop in precision indicates that the model is now overpredicting heart disease cases, leading to more false positives.
- However, test recall has significantly improved, which is crucial in medical settings. With 79% recall, the model is highly sensitive and successfully detects most actual heart disease cases.
- ROC-AUC has improved slightly and remains strong at 85%, confirming the model's ability to distinguish between individuals with and without heart disease effectively.

## 6) Inference

For inference, statsmodels was used to print out the model summary. 

In [13]:
#| echo: false
import statsmodels.api as sm

selected_features = ["Age", "GenHlth", "HighBP", "Sex", "HighChol"]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[selected_features])
X_test_scaled = scaler.transform(X_test[selected_features])

poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

feature_names = poly.get_feature_names_out(selected_features)
X_train_poly_df = pd.DataFrame(X_train_poly, columns=feature_names, index=X_train.index)
X_test_poly_df = pd.DataFrame(X_test_poly, columns=feature_names, index=X_test.index)

X_train_poly_df = sm.add_constant(X_train_poly_df)

logit_model = sm.Logit(y_train, X_train_poly_df).fit()

display(logit_model.summary())

         Current function value: 0.243818
         Iterations: 35


/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                            
================================================================================
Dep. Variable:     HeartDiseaseorAttack   No. Observations:               202944
Model:                            Logit   Df Residuals:                   202923
Method:                             MLE   Df Model:                           20
Date:                  Wed, 19 Mar 2025   Pseudo R-squ.:                  0.2191
Time:                          02:18:57   Log-Likelihood:                -49481.
converged:                        False   LL-Null:                       -63364.
Covariance Type:              nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.9044        nan        nan        nan         nan         nan
Age                  0.9512      0.018     53.533      0.000       0.916       0.986
GenHlth              0.8270      0.016     51.186      0.000       0.795       0.859
HighBP               0.5838        nan        nan        nan         nan         nan
Sex                  0.5314        nan        nan        nan         nan         nan
HighChol             0.5655        nan        nan        nan         nan         nan
Age^2               -0.0125      0.012     -1.077      0.282      -0.035       0.010
Age GenHlth         -0.1329      0.012    -11.251      0.000      -0.156      -0.110
Age HighBP          -0.1044      0.013     -8.295      0.000      -0.129      -0.080
Age Sex              0.1045      0.011      9.136      0.000       0.082       0.127
Age HighChol        -0.0355      0.012     -2.972      0.003      -0.059      -0.012
GenHlth^2            0.0100      0.007      1.345      0.179      -0.005       0.025
GenHlth HighBP      -0.0236      0.009     -2.489      0.013      -0.042      -0.005
GenHlth Sex         -0.0648      0.009     -7.558      0.000      -0.082      -0.048
GenHlth HighChol    -0.0279      0.009     -3.086      0.002      -0.046      -0.010
HighBP^2            -0.7374        nan        nan        nan         nan         nan
HighBP Sex          -0.0431      0.010     -4.504      0.000      -0.062      -0.024
HighBP HighChol      0.0408      0.009      4.292      0.000       0.022       0.059
Sex^2               -0.7771        nan        nan        nan         nan         nan
Sex HighChol         0.0313      0.009      3.497      0.000       0.014       0.049
HighChol^2          -0.7303        nan        nan        nan         nan         nan
====================================================================================
"""

**Effect of each predictor on the response** 

- **Age (0.9512)**
  - Positive coefficient means that older individuals are more likely to have heart disease.
  - Since the coefficient of Age^2 is -0.0125, this could indicate a non-linear effect where the risk increases at a faster rate at lower ages. 
  - Technical effect:
    - A one-unit increase in standardized age increases the log-odds of heart disease by 0.9512.
    - The exponent of 0.9512 is e^0.9512 = 2.59, meaning that for each unit increase in age, the odds of developing heart disease more than double.

- **General Health (0.8270)**
  - Higher general health (which indicates worse health) is strongly linked to a higher risk of heart disease.
  - The coefficient of GenHlth^2 is insignificant (p=0.179), meaning the effect is mostly linear.
  - Technical effect:
    - A one-unit increase in standardized general health (worse health) increases the log-odds of heart disease by 0.8270.
    - The exponent e^0.8270 = 2.29, meaning that each unit increase in poor health more than doubles the odds of heart disease.
    

- **High Blood Pressure (0.5838)**
  - Having higher blood pressure increases the likelihood of heart disease.
  - Since coefficient of HighBP^2 is -0.7374, there might be a diminishing effect. Individuals with extremely high blood pressure might not see exponentially increasing risk.
  - Technical effect:
    - A one-unit increase in standardized high BP increases the log-odds of heart disease by 0.5838.
    - The exponent e^0.5838 = 1.79, meaning high BP increases heart disease risk by about 79%.
    - The negative HighBP² coefficient suggests that at very high BP levels, additional risk impact slows down.

- **Sex (0.5314)**
  - Being male (Sex=1) increases the probability of heart disease compared to females (Sex=0).
  - Technical effect:
    - A one-unit increase in standardized sex (being male) increases the log-odds of heart disease by 0.5314.
    - The exponent e^0.5314 = 1.70, meaning males are 70% more likely to have heart disease than females.

- **High Cholesterol (0.5655)**
  - Having high cholesterol is positively associated with heart disease.
  - Like age and high blood pressure, HighChol^2 (-0.7303) suggests a diminishing effect at extreme cholesterol levels.
  - Technical effect:
    - A one-unit increase in standardized cholesterol levels increases the log-odds of heart disease by 0.5655.
    - The exponent e^0.5655 = 1.76, meaning high cholesterol increases heart disease risk by 76%.
    - The negative HighChol^2 coefficient suggests that beyond very high cholesterol levels, the increased risk slows down.


**Reliability**

- According to the summary table, there are 14 predictors that are statistically significant. They are considered statistically significant because their p-values are less than 0.05. 
  - Age (p = 0.000)
  - GenHlth (p = 0.000)
  - Sex (p = 0.000)
  - HighChol (p = 0.000)
  - Age × GenHlth (p = 0.000)
  - Age × HighBP (p = 0.000)
  - Age × Sex (p = 0.000)
  - Age × HighChol (p = 0.003)
  - GenHlth × HighBP (p = 0.013)
  - GenHlth × Sex (p = 0.000)
  - GenHlth × HighChol (p = 0.002)
  - HighBP × Sex (p = 0.000)
  - HighBP × HighChol (p = 0.000)
  - Sex × HighChol (p = 0.000)
- The other predictors either have p-values greater than 0.05 or returned NaN. The NaN values suggest a potential multicollinearity amongst the data set. 


**Amount of variance in the response explained by the model**

- The R-squared value in the model summary is 0.2191, indicating that 21.91% of the variation in the occurrence of heart disease is explained by the predictors included in the model. While this value may appear relatively low, it is considered reasonable in the context of medical research, where numerous unmeasured factors such as genetic predisposition, lifestyle choices, and environmental influences contribute to health outcomes. Given the complexity of heart disease occurrences, explaining 21% of the variance represents a meaningful contribution to understanding the risk factors associated with the condition.

## 7) Recommendation to Stakeholders

Based on the analysis, stakeholders should prioritize early screening and intervention for high-risk groups, particularly older individuals, those with poor general health, and individuals with high blood pressure or cholesterol. Awareness campaigns targeting men, who appear to have a higher likelihood of heart disease, could also help with preventing heart diseases. 

However, the analysis has some limitations. The dataset is highly imbalanced, with significantly more individuals without heart disease, which may bias model predictions. Additionally, the model explains only 21.91% of the variation, indicating that other unmeasured factors such as genetics, socioeconomic status, and lifestyle behaviors likely contribute to heart disease risk. Potential multicollinearity among polynomial terms could also affect model reliability.

To improve the analysis, more advanced data balancing techniques could be implemented. Additionally, incorporating more flexible and complex models could further enhance predictive accuracy and capture non-linear relationships more effectively. Finally, validating the model with external datasets will help assess its generalizability and ensure its findings can also be applied to broader populations. Addressing these limitations will lead to more accurate predictions and better-informed public health strategies to reduce heart disease risk.

## 8) Conclusion

This report identified key predictors of heart disease, and the most significant predictors were found out to be age, general health, blood pressure, cholesterol levels, and sex. While the model explains 21.91% of the variation in heart disease occurrence, its predictive power can be improved by addressing data imbalance, incorporating additional health factors, and refining feature selection. Strengthening these aspects will enhance public health interventions and support more targeted prevention strategies.